In [10]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [ ]:
df = pd.read_csv('../../lr_bulk/talon/human_talon_abundance.tsv', sep='\t')
_, _ = plot_ranked_biosamp(df,
                    sample='cell_line',
                    how='gene', 
                    nov='Known',
                    opref='figures/')

In [ ]:
df = pd.read_csv('../../lr_bulk/talon/human_talon_abundance_filtered.tsv', sep='\t')
_, _ = plot_ranked_biosamp(df,
                    sample='cell_line',
                    how='gene', 
                    nov='Known',
                    opref='figures/')

In [5]:
max_df, min_df = compute_detection(df, sample, 'gene', 'Known')

In [6]:
max_df.head()

,n_cumulative,rank,celltype
0,0,1,n/a
1,23402,2,gm12878
2,27647,3,k562
3,30204,4,h9_panc_beta
4,31342,5,h9_de


In [7]:
min_df.head()

,n_cumulative,rank,celltype
0,0,1,n/a
1,15991,2,imr90
2,19056,3,pgp1_astro
3,21192,4,hffc6
4,22808,5,h9_osteocyte


In [43]:
df = rm_sirv_ercc(df)

dataset_cols = get_sample_datasets(sample)

if how == 'iso':
    df.set_index('annot_transcript_id', inplace=True)
    df = df.loc[df.transcript_novelty == nov]
    df = df[dataset_cols]

# sum up counts across the same gene
if how == 'gene':
    # only known genes
    df = df.loc[df.gene_novelty == 'Known']
    df = df[dataset_cols+['annot_gene_id']]
    df = df.groupby('annot_gene_id').sum()

df = df.transpose()
df.reset_index(inplace=True)
df.rename({'index': 'dataset'}, axis=1, inplace=True)

In [44]:
df.head()

annot_gene_id,dataset,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.12,...,ENSG00000285981.1,ENSG00000285982.1,ENSG00000285984.1,ENSG00000285985.1,ENSG00000285986.1,ENSG00000285987.1,ENSG00000285988.1,ENSG00000285990.1,ENSG00000285991.1,ENSG00000285992.1
0,adrenal_gland_1_1,67,0,27,5,0,3,19,95,3,...,0,1,0,0,0,0,0,0,0,0
1,adrenal_gland_3_1,39,0,23,3,4,1,16,48,0,...,0,1,0,0,0,0,0,0,0,0
2,adrenal_gland_2_1,63,0,44,23,5,11,65,213,38,...,0,3,0,0,0,0,0,0,7,0
3,aorta_1_1,42,0,19,7,6,49,365,246,8,...,0,17,0,0,0,0,0,0,4,0
4,aorta_2_1,13,1,7,4,5,4,93,63,6,...,0,7,0,0,0,0,0,1,5,0


In [45]:
len(df.dataset.unique())

42

In [46]:
# # get the celltype
# if groupby == 'celltype':
#     df['celltype'] = df.dataset.str.rsplit('_', n=2, expand=True)[0]
# elif groupby == 'biosample':
#     df['celltype'] = df.dataset.str.rsplit('_', n=1, expand=True)[0]

#     print(df.head())
    
#     # add in the tissue metadata
#     fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/refs/tissue_metadata.csv'
#     tissue = pd.read_csv(fname)
#     print(tissue.head())
#     df = df.merge(tissue[['biosample', 'tissue']],
#                     how='left', left_on='celltype',
#                     right_on='biosample')
#     df.drop('celltype', axis=1, inplace=True)
#     df.rename({'tissue': 'celltype'}, axis=1, inplace=True)

# df.drop(['dataset'], axis=1, inplace=True)

In [47]:
df['celltype'] = df.dataset.str.rsplit('_', n=2, expand=True)[0]

print(df.head())

# add in the tissue metadata
fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/refs/tissue_metadata.csv'
tissue = pd.read_csv(fname)
print(tissue.head())

annot_gene_id            dataset  ENSG00000000003.14  ENSG00000000005.5  \
0              adrenal_gland_1_1                  67                  0   
1              adrenal_gland_3_1                  39                  0   
2              adrenal_gland_2_1                  63                  0   
3                      aorta_1_1                  42                  0   
4                      aorta_2_1                  13                  1   

annot_gene_id  ENSG00000000419.12  ENSG00000000457.13  ENSG00000000460.16  \
0                              27                   5                   0   
1                              23                   3                   4   
2                              44                  23                   5   
3                              19                   7                   6   
4                               7                   4                   5   

annot_gene_id  ENSG00000000938.12  ENSG00000000971.15  ENSG00000001036.13  \
0        

In [39]:
tissue.head()

,biosample_name,tissue,biosample
0,subcutaneous adipose tissue,adipose,subcutaneous_adipose_tissue
1,testis,testis,testis
2,ascending aorta,vessels,ascending_aorta
3,body of pancreas,pancreas,body_of_pancreas
4,suprapubic skin,skin,suprapubic_skin


In [40]:
df.head()

annot_gene_id,dataset,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.12,...,ENSG00000285982.1,ENSG00000285984.1,ENSG00000285985.1,ENSG00000285986.1,ENSG00000285987.1,ENSG00000285988.1,ENSG00000285990.1,ENSG00000285991.1,ENSG00000285992.1,celltype
0,adrenal_gland_1_1,67,0,27,5,0,3,19,95,3,...,1,0,0,0,0,0,0,0,0,adrenal_gland_1
1,adrenal_gland_3_1,39,0,23,3,4,1,16,48,0,...,1,0,0,0,0,0,0,0,0,adrenal_gland_3
2,adrenal_gland_2_1,63,0,44,23,5,11,65,213,38,...,3,0,0,0,0,0,0,7,0,adrenal_gland_2
3,aorta_1_1,42,0,19,7,6,49,365,246,8,...,17,0,0,0,0,0,0,4,0,aorta_1
4,aorta_2_1,13,1,7,4,5,4,93,63,6,...,7,0,0,0,0,0,1,5,0,aorta_2


In [48]:
df = df.merge(tissue[['biosample', 'tissue']],
                how='left', left_on='celltype',
                right_on='biosample')
df.drop('celltype', axis=1, inplace=True)
df.rename({'tissue': 'celltype'}, axis=1, inplace=True)

In [49]:
df.head()

,dataset,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.12,...,ENSG00000285984.1,ENSG00000285985.1,ENSG00000285986.1,ENSG00000285987.1,ENSG00000285988.1,ENSG00000285990.1,ENSG00000285991.1,ENSG00000285992.1,biosample,celltype
0,adrenal_gland_1_1,67,0,27,5,0,3,19,95,3,...,0,0,0,0,0,0,0,0,adrenal_gland,adrenal gland
1,adrenal_gland_3_1,39,0,23,3,4,1,16,48,0,...,0,0,0,0,0,0,0,0,adrenal_gland,adrenal gland
2,adrenal_gland_2_1,63,0,44,23,5,11,65,213,38,...,0,0,0,0,0,0,7,0,adrenal_gland,adrenal gland
3,aorta_1_1,42,0,19,7,6,49,365,246,8,...,0,0,0,0,0,0,4,0,aorta,vessels
4,aorta_2_1,13,1,7,4,5,4,93,63,6,...,0,0,0,0,0,1,5,0,aorta,vessels


In [50]:
# sum over celltype
df = df.groupby('celltype').sum()
temp = df.copy(deep=True)

max_df = get_rank_order(temp, 'max')
temp = df.copy(deep=True)

min_df = get_rank_order(temp, 'min')

In [9]:
max_df.head()

,n_cumulative,rank,celltype
0,0,1,n/a
1,23402,2,gm12878
2,27647,3,k562
3,30204,4,h9_panc_beta
4,31342,5,h9_de


In [10]:
min_df.head()

,n_cumulative,rank,celltype
0,0,1,n/a
1,15991,2,imr90
2,19056,3,pgp1_astro
3,21192,4,hffc6
4,22808,5,h9_osteocyte


In [11]:
df.head()

annot_gene_id,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.12,ENSG00000001167.14,...,ENSG00000285981.1,ENSG00000285982.1,ENSG00000285984.1,ENSG00000285985.1,ENSG00000285986.1,ENSG00000285987.1,ENSG00000285988.1,ENSG00000285990.1,ENSG00000285991.1,ENSG00000285992.1
celltype,,,,,,,,,,,,,,,,,,,,,
gm12878,1,0,905,90,510,783,0,948,58,153,...,0,34,1,0,1,1,0,1,0,0
h9,1280,8,322,35,296,0,0,1271,137,214,...,0,20,0,0,0,0,44,0,4,0
h9_chondro,134,0,76,25,24,0,201,384,45,79,...,0,4,0,0,0,0,0,0,23,0
h9_de,1143,13,153,71,245,1,0,850,83,282,...,0,9,0,0,0,0,3,0,0,0
h9_neural_crest,610,2,180,47,193,0,0,456,14,106,...,0,13,0,0,0,0,0,0,1,0


In [12]:
df.index.unique().tolist()

['gm12878',
 'h9',
 'h9_chondro',
 'h9_de',
 'h9_neural_crest',
 'h9_osteocyte',
 'h9_panc_beta',
 'h9_panc_progen',
 'hct116',
 'hepg2',
 'hffc6',
 'hl60',
 'imr90',
 'k562',
 'mcf7',
 'panc1',
 'pc3',
 'pgp1',
 'pgp1_astro',
 'pgp1_endo',
 'pgp1_excite_neuron']